# 16. Attention Mechanism
## 1. Attention Mechanism
* **seq2seq 모델**: 인코더에서 입력 시퀀스를 **컨텍스트 벡터**라는 하나의 고정된 크기의 벡터 표현으로 압축하고, 디코더는 이 컨텍스트 벡터를 통해서 출력 시퀀스를 만들어냄
* 하지만, 이러한 RNN 기반 seq2seq 모델은 크게 두 가지 문제가 있음
  1. 하나의 **고정**된 크기의 벡터에 **모든 정보를 압축**하려고 하니까 정보 손실 발생
  2. RNN의 고질적인 문제인 **기울기 소실(Vanishing Gratient) 문제**가 존재
* 결국 이는 기계 번역 분야에서 입력 문장이 길면 번역 품질이 떨어지는 현상으로 나타남
* 이를 위한 대안으로 입력 시퀀스가 길어지면 출력 시퀀스의 정혹도가 떨어지는 것을 보정해주기 위해 등장한 기법인 **어텐션(attention)** 소갸

### 1. The Idea of Attention
* 어텐션의 기본 아이디어: **디코더에서 출력 단어를 예측하는 매 시점(timestep)마다, 인코더에서의 전체 입력 문장을 다시 한 번 참고**한다는 점
* 단, 전체 입력 문장을 전부 다 동일한 비율로 참고하는 것이 아니라, **해당 시점에서 예측해야 할 단어와 연관이 있는 입력 단어 부분을 좀 더 집중(attention)**해서 보게 됨

### 2. Attention Function
* 우선 Key-Value로 구성되는 자료형에 대해서 잠깐 언급


In [ ]:
# 파이썬의 딕셔너리 자료형을 선언
# 키(Key): 값(Value)의 형식으로 키와 값의 쌍(Pair)를 선언
dict = {"2017": "Transformer", "2018":"BERT"}

In [ ]:
print(dict["2017"])

Transformer


In [ ]:
print(dict["2018"])

BERT


* 어텐션을 함수로 표현하면 주로 다음과 같이 표현됨
> Attention(Q, K, V) = Attention Value
* 어텐션 함수는 주어진 '쿼리(Query)'에 대해서 모든 '키(Key)'와의 유사도를 각각 구함
* 그리고 구해낸 이 유사도를 키와 맵핑되어 있는 각각의 '값(Value)'에 반영해줌
* 유사도가 반영된 '값(Value)'를 모두 더해서 리턴
* 이를 **어텐션 값(Attention Value)**라고 함
  1. Q = Query : t 시점의 디코더 셀에서의 은닉 상태 
  2. K = Keys : 모든 시점의 인코더 셀의 은닉 상태들 
  3. V = Values: 모든 시점의 인코더 셀의 은닉 상태들

### 3. Dot-Product Attention
* 소프트맥스 함수를 통해 나온 결과 값은 I, am, a, student 단어 각각이 출력 단어를 예측할 때 얼마나 도움이 되는지의 정도를 수치화한 값
####1) 어텐션 스코어(Attention score)를 구한다
* 인코더의 시점(time step)을 1, 2, ..., N이라고 하였을 때 인코더의 은닉 상태를 각각 h1, h2, .., hn이라고 하자
* 디코더의 현재 시점 t에서의 디코더의 은닉 상태를 st라고 하자
* 인코더의 은닉 상태와 디코더의 은닉 상태의 차원이 같다고 가정

* 먼저 이전 챕터에서 배웠던 **디코더의 현재 시점 t에서 필요한 입력값을 다시 상기**
  * 시점 t에서 출력 단어를 예측하기 위해서 디코더의 셀은 두 개의 입력값 필요로 하는데, 바로 이전 시점인 t-1의 은닉 상태와 이전 시점 t-1에 나온 출력 단어
* 그런데 어텐션 메커니즘에서는 출력 단어 예측에 또 다른 값을 필요로 하는데 바로 어텐션 값(Attention Value)라는 새로운 값
* t번째 단어를 예측하기 위한 어텐션 값을 at라고 정의

* 어텐션 값이 현재 시점 t에서의 출력 예측에 구체적으로 어떻게 반영되는지는 뒤에서 설명
* 지금까지 배우는 모든 과정은 at를 구하기 위한 여정
* 그 여정의 첫 걸음은 바로 **어텐션 스코어(Attention Score)**를 구하는 일
  * 정의: 현재 디코더의 시점 t에서 단어를 예측하기 위해, 인코더의 모든 은닉 상태 각각이 디코더의 현 시점의 은닉 상태 st와 얼마나 유사한지를 판단하는 스코어 값

* 닷-프로덕트 어텐션에서는 이 스코어 값을 구하기 위해 st를 전치(transpose)하고 각 은닉 상태와 내적(dot product)를 수행. 즉, 모든 어텐션 스코어 값은 스칼라
> score(st, hi) = siT hi


#### 2) Softmax function을 통해 Attention distribution을 구함
* e^t에 소프트맥스 함수를 적용하여, 모든 값을 합하면 1이 되는 확률 분포를 얻어냄
* 이를 어텐션 분포(Attention Distribution)이라고 하며, 각각의 값은 어텐션 가중치(Attention Weight)라고 함
* 디코더의 시점 t에서의 어텐션 가중치의 모음값인 어텐션 분포를 a^t라고 할 때, a^t를 식으로 정의하면 다음과 같음
> a^t = softmax(e^t)

#### 3) 각 인코더의 어텐션 가중치와 은닉 상태를 가중합하여 어텐션 값(Attention Value)를 구한다.
* 어텐션의 최종 결과값을 얻기 위해서 각 인코더의 은닉 상태와 어텐션 가중치값들을 곱하고, 최종적으로 모두 더함.
* 요약하자면 가중합(Weighted Sum)을 한다고도 말할 수 있음
* 이러한 어텐션 값 at는 종종 인코더의 문맥을 포함하고 있다고 하여 **컨텍스트 벡터(context vector)**라고도 불림
* 앞서 배운 가장 기본적인 seq2seq에서는 인코더의 마지막 은닉 상태를 컨텍스트 벡터라고 부르는 것과 대조됨

#### 4) 어텐션 값과 디코더의 t 시점의 은닉 상태를 연결(Concatenate)
* 어텐션 값이 구해지면 어텐션 메커니즘은 at를 st와 concatenate하여 하나의 벡터로 만드는 작업 수행
  * 이를 vt라고 정의
  * 그리고 이 vt를 y hat 예측 연산의 입력으로 사용하므로서 인코더로부터 얻은 정보를 활용하여 y hat을 좀 더 잘 예측할 수 있게 됨. 이것이 어텐션 메커니즘의 핵심.

#### 5) 출력층 연산의 입력이 되는 s 물결 t를 계싼
* 논문에서는 vt를 바로 출력층으로 보내기 전에 신경망 연산을 한 번 더 추가함
  * 가중치 행렬과 곱한 후에 하이퍼볼릭탄젠트 함수를 지나도록 하여 출력층 연산을 위한 새로운 벡터인 s 물결 t를 얻음
  * seq2seq에서는 출력층의 입력이 t 시점의 은닉 상태인 st였던 반면, 어텐션 메커니즘에서는 출력층의 입력이 s 물결 t가 되는 셈

#### 6) s 물결 t를 출력층의 입력으로 사용

### 4. 다양한 종류의 어텐션
* 어텐션 스코어 구하는 방법에 따라 여러 종류가 존재
  * dot, scaled dot, general, concat, location - base
* 이름이 dot이라고 붙여진 스코어 함수가 이번 챕터에서 배운 닷 프로덕트 어텐션. 이 어텐션을 제안한 사람의 이름을 따서 루옹 어텐션이라고도 함. 

## 2) 바다나우 어텐션 (Bahdanau Attention)
### 1. 바다나우 어텐션 함수 (Bahdanau Attention Function)
> Attention(Q, K, V) = Attention value
* t = 어텐션 메커니즘이 수행되는 디코더 셀의 현재 시점 의미
1. Q = Query : **t-1 시점**의 디코더 셀에서의 은닉 상태 
2. K = Keys : 모든 시점의 인코더 셀의 은닉 상태들
3. V = Values: 모든 시점의 인코더 셀의 은닉 상태들

### 2. Bahdanau Attention
* 연산 순서 이해
#### 1) 어텐션 스코어 구하기
#### 2) 소프트맥스 함수를 통해 어텐션 분포를 구하기
#### 3) 각 인코더의 어텐션 가중치와 은닉 상태를 가중합하여 어텐션 값 구하기
#### 4) 컨텍스트 벡터로부터 st를 구하기
* 기존의 LSTM: 이전 시점의 셀로부터 전달 받은 은닉 상태 s t-1와 현재 시점의 입력 x t(임베딩된 단어 벡터)를 가지고 연산
* 어텐션 메커니즘: 컨텍스트 벡터와 현재 시점의 입력 단어인 임베딩 벡터를 concatenate하고, 현재 시점의 새로운 입력으로 사용하는 모습
  * 그리고 이전 시점의 셀로부터 전달 받은 은닉 상태 s t-1와 현재 시점의 새로운 입력으로부터 s t를 구함
* 정리: 기존의 LSTM이 **임베딩된 단어 벡터를 입력**으로 하는 것에서 **컨텍스트 벡터와 임베딩된 단어 벡터를 concatenate하여 입력으로 사용**하는 것이 달라짐



## 3. 양방향 LSTM과 어텐션 메커니즘
### 1. IMDB 리뷰 데이터 전처리하기

In [2]:
from tensorflow.keras.datasets import imdb
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [3]:
vocab_size = 10000
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words = vocab_size)

17473536/17464789 [==============================] - 0s 0us/step


In [4]:
# 이미 정수 인코딩이 된 상태로 남은 전처리는 패딩뿐
print('리뷰의 최대 길이 : {}'.format(max(len(l) for l in X_train)))
print('리뷰의 평균 길이 : {}'.format(sum(map(len, X_train))/len(X_train)))

리뷰의 최대 길이 : 2494
리뷰의 평균 길이 : 238.71364


In [5]:
# 평균 길이보다 조금 더 크게 데이터 패딩
max_len = 500
X_train = pad_sequences(X_train, maxlen = max_len)
X_test = pad_sequences(X_test, maxlen = max_len)

### 2. 바다나우 어텐션
* 어텐션 스코어 함수: 주어진 query와 모든 key에 대해서 유사도를 측정
* 닷 프로덕트 어텐션: query와 key의 유사도를 구하는 방법이 내적(dot product)
  > score(query, key) = queryTkey
* 바다나우 어텐션은 아래와 같은 어텐션 스코어 함수 사용
  > score(query, key) = VTtanh(W1key + W2query(
* 텍스트 분류에서 어텐션 메커니즘 사용 이유?
  * **RNN의 마지막 은닉 상태는 예측을 위해서 사용됨**
  * **그런데 이 RNN의 마지막 은닉 상태는 몇 가지 유용한 정보들을 손실한 상태**. 그래서 RNN이 time step을 지나며 손실했던 정보들을 다시 참고하고자 함(**RNN의 모든 은닉 상태를 다시 한 번 참고**)


In [7]:
import tensorflow as tf

In [8]:
class BahdanauAttention(tf.keras.Model):
  def __init__(self, units):
    super(BahdanauAttention, self).__init__()
    self.W1 = Dense(units)
    self.W2 = Dense(units)
    self.V = Dense(1)
  
  def call(self, values, query): #단, key와 value는 같음
    # query shape == (batch_size, hidden size)
    # hidden_with_time_axis shape == (batch_size, 1, hidden size)
    # score 계산을 위해 뒤에서 할 덧셈을 위해서 차원 변경
    hidden_with_time_axis = tf.expand_dims(query, 1)

    # score shape == (batch_size, max_length 1)
    # we get 1 at the last axis because we are applying score to self.V
    # the shape of the tensor before applying self.V is (batch_size, max_length, units)
    score = self.V(tf.nn.tanh(
        self.W1(values) + self.W2(hidden_with_time_axis)))
    
    # attention_weights shape == (batch_size, max_length, 1)
    attention_weights = tf.nn.softmax(score, axis=1)

    # context_vector shape after sum == (batch_size, hidden_size)
    context_vector = attention_weights * values
    context_vector = tf.reduce_sum(context_vector, axis=1)

    return context_vector, attention_weights

### 3. 양방향 LSTM + 어텐션 메커니즘(BiLSTM with Attention Mechanism)

In [9]:
from tensorflow.keras.layers import Dense, Embedding, Bidirectional, LSTM, Concatenate, Dropout
from tensorflow.keras import Input, Model
from tensorflow.keras import optimizers
import os

In [10]:
# 모델 설계. 케라스 함수형 API 사용.
# 입력층과 임베딩층 설계

sequence_input = Input(shape=(max_len,), dtype='int32')
embedded_sequences = Embedding(vocab_size, 128, input_length=max_len, mask_zero = True)(sequence_input)

# 10000개의 단어들을 128 차원의 벡터로 임베딩하도록 설계

In [11]:
# 이제 양방향 LSTM 설계
# 여기에서는 양방향 LSTM을 두 층을 사용
# 첫번째 층. 두번째 층을 위에 쌓을 예정으로 return_sequences = True로 해줌
lstm = Bidirectional(LSTM(64, dropout = 0.5, return_sequences = True))(embedded_sequences)


In [13]:
# 두번째 층 설계
# 상태를 리턴받아야 하여 return_state = True

lstm, forward_h, forward_c, backward_h, backward_c = Bidirectional(LSTM(64, dropout=0.5, return_sequences=True, return_state=True))(lstm)

In [16]:
# 각 상태의 크기(shape)를 출력
print(lstm.shape, forward_h.shape, forward_c.shape, backward_h.shape, backward_c.shape)

# 순방향 LSTM의 은닉 상태와 셀 상태를 forward_h, forward_c에 저장하고
# 역방향 LSTM의 은닉 상태와 셀 상태를 backward_h, backward_c에 저장

# 각 은닉 상태나 셀 상태의 경우에는 128차원을 가지는데, lstm의 경우에는 (500x128) 크기를 가짐
# forward 방향과 backward 방향이 연결된 hidden state 벡터가 모든 시점에 대해서 존재함을 의미

(None, 500, 128) (None, 64) (None, 64) (None, 64) (None, 64)


In [17]:
# 양방향 LSTM을 사용할 경우에는 순방향 LSTM과 역방향 LSTM 각각 은닉 상태와 셀 상태를 가지므로
# 양방향 LSTM의 은닉 상태와 셀 상태를 사용하려면 두 방향의 LSTM의 상태들을 concatenate해야 함

state_h = Concatenate()([forward_h, backward_h]) # 은닉 상태
state_c = Concatenate()([forward_c, backward_c]) # 셀 상태

In [18]:
# 어텐션 메커니즘에서는 은닉 상태 사용
# 이를 입력으로 컨텍스트 벡터를 얻음

attention = BahdanauAttention(64) # 가중치 크기 정의
context_vector, attention_weights = attention(lstm, state_h)

In [20]:
# 컨텍스트 벡터를 밀집층(dense layer)에 통과시키고, 
# 이진 분류이므로 최종 출력층에 1개의 뉴런을 배치하고, 활성화 함수로 시그모이드 함수 사용

dense1 = Dense(20, activation="relu")(context_vector)
dropout = Dropout(0.5)(dense1)
output = Dense(1, activation="sigmoid")(dropout)
model = Model(inputs=sequence_input, outputs=output)

In [22]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [25]:
# 모델 훈련
history = model.fit(X_train, y_train, epochs = 3, batch_size = 256, validation_data=(X_test, y_test), verbose=1)

Epoch 1/3
98/98 [==============================] - 1442s 15s/step - loss: 0.4596 - accuracy: 0.7775 - val_loss: 0.3049 - val_accuracy: 0.8721
Epoch 2/3
98/98 [==============================] - 1462s 15s/step - loss: 0.2471 - accuracy: 0.9116 - val_loss: 0.2877 - val_accuracy: 0.8814
Epoch 3/3
98/98 [==============================] - 1437s 15s/step - loss: 0.1946 - accuracy: 0.9338 - val_loss: 0.3135 - val_accuracy: 0.8772


In [26]:
print("\n 테스트 정확도: %.4f" % (model.evaluate(X_test, y_test)[1]))

782/782 [==============================] - 307s 392ms/step - loss: 0.3135 - accuracy: 0.8772

 테스트 정확도: 0.8772
